In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/neural-net-nexus-2-0/sample_submission.csv
/kaggle/input/neural-net-nexus-2-0/train.csv
/kaggle/input/neural-net-nexus-2-0/test.csv


In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier


In [3]:
train = pd.read_csv("/kaggle/input/neural-net-nexus-2-0/train.csv")
test = pd.read_csv("/kaggle/input/neural-net-nexus-2-0/test.csv")


In [4]:
X = train.drop("Revenue", axis=1)
y = train["Revenue"]


In [5]:
X = pd.get_dummies(X)
test = pd.get_dummies(test)



In [6]:
X = X.fillna(X.median())
test = test.fillna(X.median())


In [7]:
X, test = X.align(test, join="left", axis=1, fill_value=0)


In [8]:
skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)


In [9]:

model = HistGradientBoostingClassifier(
    max_leaf_nodes=31,
    learning_rate=0.05,
    max_iter=400,
    early_stopping=True,
    random_state=42
)



In [10]:
scores = cross_val_score(
    model,
    X,
    y,
    cv=skf,
    scoring="accuracy"
)


In [11]:
print("Fold accuracies:", scores)
print("Mean CV accuracy:", scores.mean())

Fold accuracies: [0.90587045 0.90789474 0.89321862 0.89423077 0.90435223]
Mean CV accuracy: 0.9011133603238866


In [12]:
scores_auc = cross_val_score(
    model,
    X,
    y,
    cv=skf,
    scoring="roc_auc"
)

print("ROC-AUC scores:", scores_auc)
print("Mean ROC-AUC:", scores_auc.mean())

ROC-AUC scores: [0.93276124 0.93095692 0.91674063 0.92022291 0.93005855]
Mean ROC-AUC: 0.9261480505506686


In [13]:

model.fit(X, y)


HistGradientBoostingClassifier(early_stopping=True, learning_rate=0.05,
                               max_iter=400, random_state=42)

In [14]:
test_labels = model.predict(test)


In [15]:
sample_sub = pd.read_csv(
    "/kaggle/input/neural-net-nexus-2-0/sample_submission.csv"
)

sample_sub["Revenue"] = test_labels

In [16]:
print(sample_sub.dtypes)
print(sample_sub.head())


ID         int64
Revenue     bool
dtype: object
   ID  Revenue
0   0     True
1   1     True
2   2    False
3   3    False
4   4    False


In [17]:
sample_sub.to_csv("submission_v5.csv", index=False)



In [18]:
from joblib import dump

dump(model, "Visionaries.joblib")


['Visionaries.joblib']